In [5]:
import numpy as np
import torch
import scipy
import pandas as pd
import matplotlib.pyplot as plt
from icp import ICP_ROT
from nc_score import pose_err
from tqdm import tqdm
import tools
import rpmg
import argparse
from dataset.CameraPoseDataset import CameraPoseDatasetPred
from rpr import find_poses

In [6]:
# compute the relative pose
def normalize_vector( v):
    batch=v.shape[0]
    v_mag = torch.sqrt(v.pow(2).sum(1))# batch
    v_mag = torch.max(v_mag, torch.autograd.Variable(torch.FloatTensor([1e-8])))
    v_mag = v_mag.view(batch,1).expand(batch,v.shape[1])
    v = v/v_mag
    return v

def compute_quaternions_from_rotation_matrices(matrices):
    batch=matrices.shape[0]
    
    w=torch.sqrt(torch.max(1.0 + matrices[:,0,0] + matrices[:,1,1] + matrices[:,2,2], torch.zeros(1))) / 2.0
    w = torch.max (w , torch.autograd.Variable(torch.zeros(batch))+1e-8) #batch
    w4 = 4.0 * w
    x= (matrices[:,2,1] - matrices[:,1,2]) / w4
    y= (matrices[:,0,2] - matrices[:,2,0]) / w4
    z= (matrices[:,1,0] - matrices[:,0,1]) / w4
    quats = torch.cat((w.view(batch,1), x.view(batch, 1),y.view(batch, 1), z.view(batch, 1) ), 1   )
    quats = normalize_vector(quats)
    return quats

def compute_rotation_matrix_from_quaternion( quaternion, n_flag=True):
    batch=quaternion.shape[0]
    if n_flag:
        quat = normalize_vector(quaternion)
    else:
        quat = quaternion
    qw = quat[...,0].view(batch, 1)
    qx = quat[...,1].view(batch, 1)
    qy = quat[...,2].view(batch, 1)
    qz = quat[...,3].view(batch, 1)

    # Unit quaternion rotation matrices computatation  
    xx = qx*qx
    yy = qy*qy
    zz = qz*qz
    xy = qx*qy
    xz = qx*qz
    yz = qy*qz
    xw = qx*qw
    yw = qy*qw
    zw = qz*qw

    row0 = torch.cat((1-2*yy-2*zz, 2*xy - 2*zw, 2*xz + 2*yw), 1) #batch*3
    row1 = torch.cat((2*xy+ 2*zw,  1-2*xx-2*zz, 2*yz-2*xw  ), 1) #batch*3
    row2 = torch.cat((2*xz-2*yw,   2*yz+2*xw,   1-2*xx-2*yy), 1) #batch*3
    
    matrix = torch.cat((row0.view(batch, 1, 3), row1.view(batch,1,3), row2.view(batch,1,3)),1) #batch*3*3
    
    return matrix

def compute_relative_pose(pose1, pose2):
    '''
    pose1 and pose2 are nx7 tensors, where the first 3 elements are the translation and the last 4 elements are the quaternion.
    '''
    # Assuming `tools.compute_rotation_matrix_from_quaternion` is correctly defined elsewhere
    R1 = compute_rotation_matrix_from_quaternion(pose1[:, 3:])
    R2 = compute_rotation_matrix_from_quaternion(pose2[:, 3:])
    t1 = pose1[:, :3].unsqueeze(-1)  # Ensure t1 is nx3x1 for correct broadcasting in matrix operations
    t2 = pose2[:, :3].unsqueeze(-1)  # Ensure t2 is nx3x1
    # Compute the relative rotation
    relative_R = R2.bmm(R1.transpose(1, 2))
    
    # Compute the relative translation
    relative_t = t2 - relative_R.bmm(t1)

    # Flatten relative_t back to nx3 for concatenation
    relative_t = relative_t.squeeze(-1)

    # For returning, you may want to convert relative_R back to quaternions and concatenate with relative_t
    # Assuming `tools.compute_quaternion_from_rotation_matrix` is a function that converts rotation matrices to quaternions
    relative_quaternions = compute_quaternions_from_rotation_matrices(relative_R)
    relative_pose = torch.cat((relative_t, relative_quaternions), dim=1)

    return relative_pose

def compute_and_compare_pose(T_r, test_pose, gt_pose):
    """
    Compute the final pose using the relative pose T_r and test_pose, 
    and compare it with the ground truth pose (gt_pose).
    
    :param T_r: numpy array of shape (4, 4), the relative transformation matrix
    :param test_pose: torch.Tensor of shape (1, 7), the test pose in translation + quaternion format
    :param gt_pose: numpy array of shape (7,), the ground truth pose in translation + quaternion format
    :return: position error and orientation error
    """
    # Convert quaternions to rotation matrices
    R_test = compute_rotation_matrix_from_quaternion(test_pose[:, 3:], n_flag=True)

    # Create transformation matrices for test_pose
    T_test = torch.zeros((4, 4))
    T_test[:3, :3] = R_test.squeeze(0)
    T_test[:3, 3] = test_pose[:, :3].squeeze(0)
    T_test[3, 3] = 1
    
    # Convert numpy T_r to tensor and apply relative transformation
    T_final = torch.mm(torch.FloatTensor(T_r), T_test)

    # Convert the final transformation matrix back to translation and quaternion format
    t_final = T_final[:3, 3]
    R_final = T_final[:3, :3]
    quaternion_final = compute_quaternions_from_rotation_matrices(R_final.unsqueeze(0))

    # Final pose in translation + quaternion format
    final_pose = torch.cat((t_final, quaternion_final.squeeze(0)), 0).unsqueeze(0)

    # Compare with gt_pose
    gt_pose_tensor = torch.FloatTensor(gt_pose).unsqueeze(0)
    posit_err, orient_err = pose_err(final_pose, gt_pose_tensor)

    return posit_err.item(), orient_err.item()

In [7]:
from dataset.CameraPoseDataset import CameraPoseDatasetPred
dataset_path = '/home/runyi/Data/7Scenes'
cal_labels_file = '/home/runyi/Project/TBCP6D/dataset/7Scenes_0.5/abs_7scenes_pose.csv_chess_cal.csv_results.csv'
test_labels_file = '/home/runyi/Project/TBCP6D/dataset/7Scenes_0.5/abs_7scenes_pose.csv_chess_test.csv_results.csv'
cal_set = CameraPoseDatasetPred(dataset_path, cal_labels_file)
test_set = CameraPoseDatasetPred(dataset_path, test_labels_file)

In [8]:
cal_set.poses.shape, cal_set.pred_poses.shape

((2417, 7), (2417, 7))

In [9]:
icp_rot = ICP_ROT(torch.tensor(cal_set.poses[:, 3:]), torch.tensor(cal_set.pred_poses[:, 3:]))
icp_rot.compute_non_conformity_scores()

tensor([0.3534, 1.0696, 1.6345,  ..., 1.8173, 2.1381, 3.8093],
       dtype=torch.float64)

In [10]:
dataloader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False, num_workers=1)

In [11]:
import cv2

# Function to find and match SURF features
def find_and_match_features(image1, image2):
    # Create SIFT object
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(image1, None)
    kp2, des2 = sift.detectAndCompute(image2, None)
    
    # BFMatcher with default params
    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)
    
    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75*n.distance:
            good_matches.append(m)
    
    return kp1, kp2, good_matches

# Function to estimate essential matrix and recover pose
def estimate_pose(kp1, kp2, matches, K):
    points1 = np.float32([kp1[m.queryIdx].pt for m in matches])
    points2 = np.float32([kp2[m.trainIdx].pt for m in matches])
    E, mask = cv2.findEssentialMat(points1, points2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)
    _, R, t, mask = cv2.recoverPose(E, points1, points2, K)
    return R, t

def find_poses(image1, image2):
    kp1, kp2, good_matches = find_and_match_features(image1, image2)
    K = np.array([[585, 0, 320],
                  [0, 585, 240],
                  [0, 0, 1]], dtype=np.float32)
    R, t = estimate_pose(kp1, kp2, good_matches, K)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = t.squeeze()  # Ensure t is correctly shaped
    return T



In [12]:
from tqdm.autonotebook import tqdm
for idx, minibatch in enumerate(dataloader):
    img = minibatch.get('img')[0].numpy()  # Assuming this retrieves the current image
    pred_pose = minibatch.get('est_pose')  # Predicted pose from the model
    # Convert quaternion part to rotation matrix
    pred_R = compute_rotation_matrix_from_quaternion(pred_pose[:, 3:]).squeeze(0)
    pred_t = pred_pose[:, :3].squeeze(0)
    
    pose_errs0, pose_errs1, pose_errs2, pose_errs3, pose_errs4 = [], [], [], [], []
    
    for idx, cal_img in tqdm(enumerate(cal_set.imgs)):
        # Assume find_poses computes the relative transformation matrix correctly
        relative_pose = torch.tensor(find_poses(img, cal_img))
        relative_R = relative_pose[:3, :3]
        relative_t = relative_pose[:3, 3]
        # cal gt 
        # relative pose
        # test pred pose
        adjusted_R = relative_R.T @ pred_R
        
        adjusted_q = compute_quaternions_from_rotation_matrices(adjusted_R.unsqueeze(0))
        adjusted_t = relative_R.T @ pred_t - relative_R.T @ relative_t
        adjusted_v = torch.cat((adjusted_t, adjusted_q.squeeze(0)), 0).unsqueeze(0)
        
        adj_t_err, adj_o_err = pose_err(adjusted_v, torch.tensor(cal_set.poses[idx]).unsqueeze(0))
        p0, o0 = pose_err(pred_pose, torch.tensor(cal_set.poses[idx]).unsqueeze(0))
        print(p0.item(), adj_t_err.item(), o0.item(), adj_o_err.item())
    #     pose_errs0.append(o0.item()); pose_errs1.append(orient_err1.item()); pose_errs2.append(orient_err2.item()); pose_errs3.append(orient_err3.item()); pose_errs4.append(orient_err4.item())
    #     if idx == 200:
    #         break
    # print(np.mean(pose_errs0), np.mean(pose_errs1), np.mean(pose_errs2), np.mean(pose_errs3), np.mean(pose_errs4))
    break
    #     relative_R = relative_pose[:3, :3]
    #     relative_t = relative_pose[:3, 3]
    #     # Compute the adjusted pose
    #     adjusted_R = pred_R @ relative_R.T
    #     adjusted_t = relative_R.bmm(pred_t.unsqueeze(-1)) + relative_t.unsqueeze(-1)
    #     adjusted_quaternions = compute_quaternions_from_rotation_matrices(adjusted_R)
    #     adjusted_pose_vector = torch.cat((adjusted_t.squeeze(2), adjusted_quaternions), dim=1)
        
    #     cal_gt_pose = torch.tensor(cal_set.poses[idx], dtype=torch.float64).unsqueeze(0)
    #     # Compute positional and orientation errors
    #     posit_err, orient_err = pose_err(adjusted_pose_vector, cal_gt_pose)
    #     print(pose_err(torch.tensor(pred_pose).to(torch.float64), cal_gt_pose))
    #     print(posit_err.item(), orient_err.item()) 
    

0it [00:00, ?it/s]

0.468002795298486 0.6876909731023402 5.612451309633198 3.9345160974890665
0.47171771092498554 0.6292939996193537 5.472318325396608 2.767206653701752
0.47180939790871196 0.6234745133167432 5.633418987270219 2.8004720421336002
0.4726175504811338 0.6712018291125184 5.61653517336346 3.6504108245301534
0.4733220965802733 0.6499754370931642 5.62926274980952 3.34443880677191
0.4715396107964508 0.6662515344951502 5.611286257145665 3.564053990532679
0.47203121821776445 0.6387795159132522 5.524266570993478 2.971067552047768
0.4724456047111646 0.7312202906345413 5.359687957273786 4.718441595996223
0.4690501716699451 0.6435635435119023 5.154753107166995 3.317676738971387
0.4622852165348679 0.6464069169862703 5.428869515220478 3.056308445697282
0.4581219864092347 0.6667712969001964 5.494086120752482 3.8200028954936385
0.45191864463792986 0.5919539396348936 5.1294849563963805 1.89146174880846
0.44845797901781687 0.6478982642848345 5.252769137110474 3.694513419880115
0.44583624822807855 0.68882669148

KeyboardInterrupt: 

In [51]:
icp_rot = ICP_ROT(torch.tensor(cal_set.poses[:, 3:]), torch.tensor(cal_set.pred_poses[:, 3:]))

In [52]:
icp_rot.compute_non_conformity_scores()

tensor([0.3534, 1.0696, 1.6345,  ..., 1.8173, 2.1381, 3.8093],
       dtype=torch.float64)

In [67]:
import numpy as np

percentile_95 = np.percentile(icp_rot.non_conformity_scores, 50)
percentile_95

1.9584651125763513

In [2]:
import cv2
cv2.xfeatures2d.SURF_create(400)

error: OpenCV(4.9.0) /io/opencv_contrib/modules/xfeatures2d/src/surf.cpp:1026: error: (-213:The function/feature is not implemented) This algorithm is patented and is excluded in this configuration; Set OPENCV_ENABLE_NONFREE CMake option and rebuild the library in function 'create'


In [59]:
test_pred_pose

array([-0.0316476 , -0.76844001, -0.69450545,  0.99021727, -0.13535483,
        0.02634406, -0.02132878])